# 8.3 Objectives

In [1]:
# install dependencies
%pip install -q amplpy

from amplpy import AMPL, ampl_notebook

ampl = ampl_notebook(
    modules=['highs'],  # modules to install
    license_uuid='default',  # license to use
)  # instantiate AMPL object and register magics

Note: you may need to restart the kernel to use updated packages.


The declaration of an objective function consists of one of the keywords `minimize`
or `maximize`, a name, a colon, and an expression in previously defined sets,
parameters, and variables. We have seen examples such as

```ampl
minimize Total_Cost: sum {j in FOOD} cost[j] * Buy[j];
```

and

```ampl
maximize Total_Profit:
  sum {p in PROD, t in 1..T}
    ( sum {a in AREA[p]} revenue[p,a,t] * Sell[p,a,t]
    - prodcost[p] * Make[p,t]
    - invcost[p] * Inv[p,t] );
```

The name of the objective plays no further role in the model, with the exception of
certain *columnwise* declarations to be introduced in Chapters 15 and 16 xTODO. Within
AMPL commands, the objective's name refers to its value. Thus, for example, after
solving a feasible instance of the diet model in
[Figure 2-1](../02/2_2_an_AMPL_model_for_the_diet_problem.ipynb#fig-2-1),
we could issue the command

```ampl
ampl.display('{j in FOOD} 100 * cost[j] * Buy[j] / Total_Cost')
```

which produces

```text
100*cost[j]*Buy[j]/Total_Cost [*] :=
BEEF 14.4845
CHK   4.38762
FISH  3.8794
HAM  24.4792
MCH  16.0089
MTL  16.8559
SPG  15.6862
TUR   4.21822
;
```

to show the percentage of the total cost spent on each food.

Although a particular linear program must have one objective function, a model may
contain more than one objective declaration. Moreover, any `minimize` or `maximize`
declaration may define an indexed collection of objective functions, by including
an indexing expression after the objective name. In these cases, you may indicate the objective name as the `problem=` argument in the `solve()` function.

As an example, recall that when trying to solve the model of
[Figure 2-1](../02/2_2_an_AMPL_model_for_the_diet_problem.ipynb#fig-2-1)
with the accompanying data, we found that no solution could satisfy all of the
constraints. We subsequently increased the sodium (`NA`) limit to 50000 to make a
feasible solution possible.

In [2]:
%%writefile diet.mod

set NUTR;
set FOOD;

param cost {FOOD} > 0;
param f_min {FOOD} >= 0;
param f_max {j in FOOD} >= f_min[j];

param n_min {NUTR} >= 0;
param n_max {i in NUTR} >= n_min[i];

param amt {NUTR,FOOD} >= 0;

var Buy {j in FOOD} >= f_min[j], <= f_max[j];

minimize Total_Cost:  sum {j in FOOD} cost[j] * Buy[j];

subject to Diet {i in NUTR}:
   n_min[i] <= sum {j in FOOD} amt[i,j] * Buy[j] <= n_max[i];

Overwriting diet.mod


In [3]:
import pandas as pd

df_food2 = pd.DataFrame(
    [
        ['BEEF', 3.19, 2, 10],
        ['CHK', 2.59, 2, 10],
        ['FISH', 2.29, 2, 10],
        ['HAM', 2.89, 2, 10],
        ['MCH', 1.89, 2, 10],
        ['MTL', 1.99, 2, 10],
        ['SPG', 1.99, 2, 10],
        ['TUR', 2.49, 2, 10]
    ],
    columns = ['FOOD', 'cost', 'f_min', 'f_max']
).set_index('FOOD')

df_nutr2 = pd.DataFrame(
    [
        ['A', 700, 20000],
        ['C', 700, 20000],
        ['B1', 700, 20000],
        ['B2', 700, 20000],
        ['NA', 0, 40000],
        ['CAL', 16000, 24000],
    ],
    columns = ['NUTR', 'n_min', 'n_max']
).set_index('NUTR')

df_amt2 = pd.DataFrame(
    [
        ['BEEF', 60, 20, 10, 15, 938, 295],
        ['CHK', 8, 0, 20, 20, 2180, 770],
        ['FISH', 8, 10, 15, 10, 945, 440],
        ['HAM', 40, 40, 35, 10, 278, 430],
        ['MCH', 15, 35, 15, 15, 1182, 315],
        ['MTL', 70, 30, 15, 15, 896, 400],
        ['SPG', 25, 50, 25, 15, 1329, 370],
        ['TUR', 60, 20, 15, 10, 1397, 450]
    ],
    columns = ['FOOD', 'A', 'C', 'B1', 'B2', 'NA', 'CAL']
).set_index('FOOD').T

It is reasonable to ask: *How much of an increase in the sodium limit is really
necessary to permit a feasible solution?* For this purpose we can introduce a new
objective equal to the total sodium in the diet:

```ampl
minimize Total_NA: sum {j in FOOD} amt["NA", j] * Buy[j];
```

(We create this objective only for sodium, because we have no reason to minimize most
of the other nutrients.)

In [4]:
%%writefile sodium_obj.mod

minimize Total_NA: sum {j in FOOD} amt["NA", j] * Buy[j];

Writing sodium_obj.mod


We are adding an extension to the model with this new objective and save it in `sodium_obj.mod`. AMPL can read several model files sequentially, so first we will read `diet.mod`, and then `sodium_obj.mod`. In practice, we could have merged the two objectives in the same model file.

We can solve the linear program for total cost as before, since AMPL chooses the model's first objective by default:

In [5]:
ampl = AMPL()
ampl.read('diet.mod')
ampl.read('sodium_obj.mod')

ampl.set_data(df_food2, 'FOOD')
ampl.set_data(df_nutr2, 'NUTR')
ampl.param['n_max']["NA"] = 50000
ampl.param['amt'] = df_amt2

ampl.solve(solver='highs')

HiGHS 1.11.0: optimal solution; objective 118.0594032
5 simplex iterations
0 barrier iterations
Objective = Total_Cost


The solver reports objective 118.0594032.

The solver tells us the minimum cost, and we can also use `display` to look at the
total sodium, even though it is not currently being minimized:

In [6]:
ampl.display('Total_NA')

Total_NA = 50000



Next, we use the `problem` argument in `solve()` to switch the objective to minimization of total
sodium. AMPL re-optimizes with this alternative objective, and we
display `Total_Cost` to determine the resulting cost:

In [7]:
ampl.solve(problem='Total_NA')
ampl.display('Total_Cost')

HiGHS 1.11.0: optimal solution; objective 48186
2 simplex iterations
0 barrier iterations
Total_Cost = 123.627



We see that sodium can be brought down by about 1 800 units, though the cost is forced
up by about $5.50 as a result. (Healthier diets are in general more expensive, because
they force the solution away from the one that minimizes cost.)

As another example, consider experimenting with different optimal solutions for the
office assignment problem. First we solve the original problem:

In [8]:
%%writefile transp.mod

set ORIG;   # origins
set DEST;   # destinations

param supply {ORIG} >= 0;   # amounts available at origins
param demand {DEST} >= 0;   # amounts required at destinations

   check: sum {i in ORIG} supply[i] = sum {j in DEST} demand[j];

param cost {ORIG,DEST} >= 0;   # shipment costs per unit
var Trans {ORIG,DEST} >= 0;    # units to be shipped

minimize Total_Cost:
   sum {i in ORIG, j in DEST} cost[i,j] * Trans[i,j];

subject to Supply {i in ORIG}:
   sum {j in DEST} Trans[i,j] = supply[i];

subject to Demand {j in DEST}:
   sum {i in ORIG} Trans[i,j] = demand[j];

Overwriting transp.mod


In [9]:
ORIG = ['Coullard', 'Daskin', 'Hazen', 'Hopp', 'Iravani', 'Linetsky', 'Mehrotra', 'Nelson', 'Smilowitz', 'Tamhane', 'White']

DEST = ['C118', 'C138', 'C140', 'C246', 'C250', 'C251', 'D237', 'D239', 'D241', 'M233', 'M239']

# List of "ORIG" number of elements, all of them being 1
supply = [1] * len(ORIG)

# List of "DEST" number of elements, all of them being 1
demand = [1] * len(DEST)

df_cost = pd.DataFrame(
    [
        ['Coullard', 6, 9, 8, 7, 11, 10, 4, 5, 3, 2, 1],
        ['Daskin', 11, 8, 7, 6, 9, 10, 1, 5, 4, 2, 3],
        ['Hazen', 9, 10, 11, 1, 5, 6, 2, 7, 8, 3, 4],
        ['Hopp', 11, 9, 8, 10, 6, 5, 1, 7, 4, 2, 3],
        ['Iravani', 3, 2, 8, 9, 10, 11, 1, 5, 4, 6, 7],
        ['Linetsky', 11, 9, 10, 5, 3, 4, 6, 7, 8, 1, 2],
        ['Mehrotra', 6, 11, 10, 9, 8, 7, 1, 2, 5, 4, 3],
        ['Nelson', 11, 5, 4, 6, 7, 8, 1, 9, 10, 2, 3],
        ['Smilowitz', 11, 9, 10, 8, 6, 5, 7, 3, 4, 1, 2],
        ['Tamhane', 5, 6, 9, 8, 4, 3, 7, 10, 11, 2, 1],
        ['White', 11, 9, 8, 4, 6, 5, 3, 10, 7, 2, 1]
    ],
    columns=['ORIG', 'C118', 'C138', 'C140', 'C246', 'C250', 'C251', 'D237', 'D239', 'D241', 'M233', 'M239']
).set_index('ORIG')

In [10]:
ampl = AMPL()
ampl.read('transp.mod')
ampl.set['ORIG'] = ORIG
ampl.set['DEST'] = DEST
ampl.param['supply'] = supply
ampl.param['demand'] = demand
ampl.param['cost'] = df_cost
ampl.solve(solver='highs')

HiGHS 1.11.0: optimal solution; objective 28
19 simplex iterations
0 barrier iterations


We then adjust display options and examine the solution:

In [11]:
ampl.option['display_1col'] = 1000
ampl.option['omit_zero_rows'] = 1
ampl.option['display_eps'] = 0.000001

ampl.display('Total_Cost, {i in ORIG, j in DEST} cost[i,j] * Trans[i,j]');

Total_Cost = 28

cost[i,j]*Trans[i,j] :=
Coullard  D241   3
Daskin    D237   1
Hazen     C246   1
Hopp      C251   5
Iravani   C138   2
Linetsky  C250   3
Mehrotra  D239   2
Nelson    C140   4
Smilowitz M233   1
Tamhane   C118   5
White     M239   1
;



```text
Total_Cost = 28
cost[i,j]*Trans[i,j] :=
Coullard C118    6
Daskin    D241   4
Hazen     C246   1
Hopp      D237   1
Iravani   C138   2
Linetsky C250    3
Mehrotra D239    2
Nelson    C140   4
Smilowitz M233   1
Tamhane   C251   3
White     M239   1
;
```

To keep the objective value at this optimal level while we experiment, we add a
constraint that fixes the objective expression at its current value, 28:

In [12]:
%%writefile additional_constraints.mod

subject to Stay_Optimal:
    sum {i in ORIG, j in DEST} cost[i,j] * Trans[i,j] = 28;

Overwriting additional_constraints.mod


Recall that `cost[i,j]` is the ranking that person `i` has given to office `j`, while
`Trans[i,j]` equals 1 if person `i` is assigned to office `j`, and 0 otherwise. Thus

```ampl
sum {j in DEST} cost[i,j] * Trans[i,j]
```

always equals the ranking of person `i` for the office to which that person is
assigned. We use this expression to declare a new indexed objective function:

In [13]:
%%writefile preference_objectives.mod

minimize Pref_of {i in ORIG}:
    sum {j in DEST} cost[i,j] * Trans[i,j];

Overwriting preference_objectives.mod


This statement creates, for each person `i`, an objective `Pref_of[i]` that minimizes
the ranking of the office assigned to that person. We can then select one individual
and optimize their ranking:

In [14]:
# Loading Stay_Optimal constraints
ampl.read('additional_constraints.mod')
# Loading Pref_of objectives
ampl.read('preference_objectives.mod')
ampl.solve(problem='Pref_of["Coullard"]')

HiGHS 1.11.0: optimal solution; objective 3
24 simplex iterations
0 barrier iterations


```text
HiGHS 1.11.0: optimal solution; objective 3
24 simplex iterations
0 barrier iterations
```

Examining the new assignment, we see that the original objective value is unchanged,
while the selected individual's assignment has improved, necessarily at the expense
of others:

In [15]:
ampl.display('Total_Cost, {i in ORIG, j in DEST} cost[i,j] * Trans[i,j]');

Total_Cost = 28

cost[i,j]*Trans[i,j] :=
Coullard  D241   3
Daskin    D237   1
Hazen     C246   1
Hopp      C251   5
Iravani   C138   2
Linetsky  C250   3
Mehrotra  D239   2
Nelson    C140   4
Smilowitz M233   1
Tamhane   C118   5
White     M239   1
;



We were able to make this change because there are multiple optimal solutions to
the original total-ranking objective. A solver arbitrarily returns one of these, but by
introducing a second objective we can guide it toward others.

## Multi-objective problems

Many real-world problems have multiple objectives; often this scenario is tackled by blending all the objectives by linear combination when formulating the model, or by minimizing unwanted objective deviations from a pre-specified goal.

Alternatively, if some constraints are ‘soft’, they can be modeled by penalties included in the primary or secondary objectives.

To consider multiple objectives in an AMPL model, please consider looking into the [Multiple objectives documentation](https://mp.ampl.com/modeling-mo.html).
